### 1. Setup your .env file and install the requirements

Create an .env file using the provided env.example file, and add your OpenAI API key

Then, you can run the following cell to install the library requirements:

In [ ]:
pip install -r requirements.txt

### 2. Import the libraries

In [2]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

True

### 3. Create ToT prompt chain

Step 1: Thought Decomposition (breaking down the problem)

The goal in this step is to devide the prom into manegeable subproblems, each solved by generating a "thought".

In [34]:
template = """
Step1:

I'm facing the following challenge: 

{input}

Could you help me brainstorm several diverse solutions?
In your considerations, I encourage you to factor in a range of influential elements, such as {key_drivers}.

Answer:
"""

prompt = PromptTemplate(
    input_variables=["input", "key_drivers"],
    template = template
)

chain1 = LLMChain(
    llm=ChatOpenAI(temperature=0.5, model_name="gpt-3.5-turbo"),
    prompt=prompt,
    output_key="possible_solutions"
)


Step 2: Thought Generator (creating potential solutions)

The goal is to generate potential "thoughts" to be added to the current state (sequence of thoughts)

In [35]:
template = """
Step 2:

Weigh each solution's strengths, weaknesses, startup costs, implementation hurdles, potential roadblocks, and expected results. Then, assign a success probability and confidence level based on this analysis.

Answer: {possible_solutions}
"""

prompt = PromptTemplate(
    input_variables=["possible_solutions"],
    template = template
)

chain2 = LLMChain(
    llm=ChatOpenAI(temperature=0.5, model_name="gpt-3.5-turbo"),
    prompt=prompt,
    output_key="assessment"
)


Step 3: State Evaluator (assessing potetial)

The goal it to evaluate the quality and potential of different states (sequences of thoughts) for solving the problem.

In [36]:
template = """
Step 3:

Dive deep into each solution!
Imagine different scenarios it might create, plot out implementation strategies, identify crucial aspects and resources, and brainstorm ways to tackle potential problems.
Don't forget to consider surprise twists and turns, and how we'd weather them gracefully.

Answer: {assessment}
"""

prompt = PromptTemplate(
    input_variables=["assessment"],
    template = template
)

chain3 = LLMChain(
    llm=ChatOpenAI(temperature=0.5, model_name="gpt-3.5-turbo"),
    prompt=prompt,
    output_key="intensify_reflection"
)


Step 4: Search Algorithm (finding the best solution)

The goal is to explore the search space of possible states (thought tree) and find the optimal solution.

In [37]:
template = """
Step 4:
Armed with our evaluations and scenario simulations, let's rank these solutions! Each ranking demands a rock-solid justification.
And before we crown the champion, let's give each contender a final once-over, unearthing any lingering doubts or hidden potential.

Answer: {intensify_reflection}
"""

prompt = PromptTemplate(
    input_variables=["intensify_reflection"],
    template=template
)

chain4 = LLMChain(
    llm=ChatOpenAI(temperature=0.5, model_name="gpt-3.5-turbo"),
    prompt=prompt,
    output_key="graded_possibilities"
)


Put the overall chain together. We'll employ the 'SequentialChain' component to seamlessly integrate the four chains, ensuring that the output of one chain serves as the input for the subsequent chain. 

In [38]:
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[chain1, chain2, chain3, chain4],
    input_variables=["input", "key_drivers"],
    output_variables=["graded_possibilities"],
    verbose=True
)

input = """
A city's government is facing challenges with its current transportation system, which is causing increased traffic congestion, pollution, and dissatisfaction among residents. The city aims to transition to a more sustainable, efficient, and user-friendly transportation system. This project involves multiple stakeholders, including city planners, transportation companies, environmental groups, and the public.

I need to develop a comprehensive plan for a sustainable urban transportation system that reduces congestion and pollution, improves accessibility and convenience for all residents, and is economically viable and scalable.
"""

answer = overall_chain.invoke({
    "input": input,
    "key_drivers": "infrastructure, traffic, public transit, non-motorized transport, policy and regulation, community engagement"
})




> Entering new SequentialChain chain...

> Finished chain.


In [39]:
print(answer['graded_possibilities'])

Ranking of solutions:

1. Non-motorized transport options: This solution not only addresses transportation issues but also promotes healthier lifestyles and reduces carbon emissions. It is a sustainable and cost-effective option that can have a significant impact on the overall urban environment.

2. Smart traffic management systems: Implementing smart traffic management systems can greatly improve traffic flow, reduce travel times, and enhance road safety. This solution is practical and efficient in optimizing transportation systems.

3. Comprehensive public transportation system: While expanding public transportation networks is essential for reducing traffic congestion and emissions, it may require significant investment and time to implement. However, it remains a crucial aspect of sustainable urban transportation.

4. Carpooling and ridesharing initiatives: Encouraging carpooling and ridesharing can help reduce the number of single-occupancy vehicles on the road, but it may face c